In [1]:
import pandas as pd
from google.colab import files

def upload_file():
    uploaded = files.upload()
    if uploaded:
        for filename in uploaded.keys():
            return filename
    return None

def read_sensor_data(file_path):
    # Read the file, using the second row for frequency
    df = pd.read_csv(file_path, skiprows=1, header=None)
    frequency = df.iloc[0, 0]
    return df.drop(0), frequency

def normalize_data(sensor_df, sensor_freq, target_freq):
    chunk_size = int(sensor_freq / target_freq)
    return sensor_df.groupby(sensor_df.index // chunk_size).mean()

def merge_and_save(eda_df, hr_df, temp_df, eda_freq, hr_freq, temp_freq):
    target_freq = min(eda_freq, hr_freq, temp_freq)
    if eda_freq > target_freq:
        eda_df = normalize_data(eda_df, eda_freq, target_freq)
    if temp_freq > target_freq:
        temp_df = normalize_data(temp_df, temp_freq, target_freq)
    merged_df = pd.concat([eda_df.reset_index(drop=True), hr_df.reset_index(drop=True), temp_df.reset_index(drop=True)], axis=1)

    # Label columns as EDA, HR, Temp
    merged_df.columns = ["EDA", "HR", "Temp"]

    # merged_df.to_csv('normalized_sensor_data.csv', index=False)
    # files.download('normalized_sensor_data.csv')
    return merged_df

# Upload files
print("Upload EDA file:")
eda_file = upload_file()
print("Upload HR file:")
hr_file = upload_file()
print("Upload TEMP file:")
temp_file = upload_file()

# Read and process data
eda_df, eda_freq = read_sensor_data(eda_file)
hr_df, hr_freq = read_sensor_data(hr_file)
temp_df, temp_freq = read_sensor_data(temp_file)

# Normalize and merge data, then save to a new CSV with labeled columns
merged_df = merge_and_save(eda_df, hr_df, temp_df, eda_freq, hr_freq, temp_freq)

Upload EDA file:


Saving EDA.csv to EDA.csv
Upload HR file:


Saving HR.csv to HR.csv
Upload TEMP file:


Saving TEMP.csv to TEMP.csv


In [2]:
import pandas as pd

def enhance_data(df):
    # Calculate statistical measures for EDA, HR, and Temp for each row
    df['EDA_mean'] = df['EDA'].expanding().mean()
    df['EDA_std'] = df['EDA'].expanding().std()
    df['EDA_min'] = df['EDA'].expanding().min()
    df['EDA_max'] = df['EDA'].expanding().max()

    df['HR_mean'] = df['HR'].expanding().mean()
    df['HR_std'] = df['HR'].expanding().std()
    df['HR_min'] = df['HR'].expanding().min()
    df['HR_max'] = df['HR'].expanding().max()

    df['Temp_mean'] = df['Temp'].expanding().mean()
    df['Temp_std'] = df['Temp'].expanding().std()
    df['Temp_min'] = df['Temp'].expanding().min()
    df['Temp_max'] = df['Temp'].expanding().max()

    return df

# Load the combined_final.csv
#df = pd.read_csv('/content/normalized_sensor_data.csv')

# Enhance the data
enhanced_df = enhance_data(merged_df)

# Drop rows with missing values
enhanced_df = enhanced_df.dropna()

# Save the enhanced DataFrame to a new CSV file
enhanced_df.to_csv('enhanced_combined_final.csv', index=False)